In [1]:
# from gatenlp import Document
# from gatenlp.gateworker import GateWorker

# from gatenlp.gateworker import GateWorkerAnnotator
# from gatenlp.processing.executor import SerialCorpusExecutor

import os

import unicodedata
from vncorenlp import VnCoreNLP
import re
from nltk import sent_tokenize

#for module import
import sys, os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from shared_functions.global_functions import *
from shared_functions.gg_sheet import *

In [ ]:
from transformers import AutoTokenizer, AutoModel

# PhoBERT base model
model_name = "vinai/phobert-base"

# Load tokenizer & model (first run will download to ~/.cache/huggingface)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

print("PhoBERT downloaded and ready!")

In [2]:
# To perform word segmentation, POS tagging, NER and then dependency parsing
# annotator1 = VnCoreNLP("D:/Study/Education/Projects/Group_Project/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

ner_annotator = VnCoreNLP("D:/Study/Education/Projects/Group_Project/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g') 
# To perform word segmentation and then POS tagging
# annotator = VnCoreNLP("", annotators="wseg,pos", max_heap_size='-Xmx2g') 
# To perform word segmentation only
# annotator = VnCoreNLP("", annotators="wseg", max_heap_size='-Xmx500m') 

2025-10-06 14:37:56,137|INFO|vncorenlp.vncorenlp|Starting server on: http://127.0.0.1:65095
2025-10-06 14:37:56,153|INFO|vncorenlp.vncorenlp|Server ID: 12488
2025-10-06 14:37:58,202|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:38:05,250|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:38:12,282|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:38:19,332|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:38:26,381|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:38:33,425|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:38:40,474|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:38:47,501|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:38:54,414|INFO|vncorenlp.vncorenlp|The server is now available on: http://127.0.0.1:65095


In [3]:
from underthesea import ner as uts_ner

rdrsegmenter = VnCoreNLP(
    "D:/Study/Education/Projects/Group_Project/VnCoreNLP/VnCoreNLP-1.1.1.jar", 
    annotators="wseg,pos,ner,parse", 
    max_heap_size='-Xmx2g'
)

2025-10-06 14:40:10,076|INFO|vncorenlp.vncorenlp|Starting server on: http://127.0.0.1:52889
2025-10-06 14:40:10,111|INFO|vncorenlp.vncorenlp|Server ID: 12684
2025-10-06 14:40:12,172|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:40:19,203|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:40:26,256|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:40:33,294|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:40:40,318|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:40:47,350|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:40:54,397|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:41:01,430|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:41:08,465|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-06 14:41:15,512|INFO|vnc

In [4]:
import re
import pandas as pd

import pandas as pd

def annotate_legal_entities(text):
    """
    Perform unified legal entity annotation with:
      - VNCoreNLP
      - Underthesea
    Returns: DataFrame with tokens and final NER label.
    """
    # === 1️⃣ Run VNCoreNLP ===
    annotated = rdrsegmenter.annotate(text)
    tokens = [tok["form"].replace("_", " ") for tok in annotated["sentences"][0]]

    vn_data = [
        {
            "form": tok["form"].replace("_", " "),
            "pos": tok["posTag"],
            "vncorenlp_ner": tok["nerLabel"],
            "underthesea_ner": "O",  # placeholder
            "final_label": "O"        # placeholder
        }
        for tok in annotated["sentences"][0]
    ]

    df = pd.DataFrame(vn_data)
    df["index"] = range(1, len(df) + 1)

    # === 2️⃣ Underthesea NER ===
    uts_entities = uts_ner(text)  # returns list of (entity_text, start, end, tag)
    for ent in uts_entities:
        entity_text, start, end, tag = ent
        # mark tokens that belong to this entity
        for i, tok in enumerate(tokens):
            if tok in entity_text:
                df.loc[i, "underthesea_ner"] = tag

    # === 3️⃣ Merge VNCoreNLP and Underthesea ===
    def merge_labels(row):
        if row["vncorenlp_ner"] != "O":
            return row["vncorenlp_ner"]
        elif row["underthesea_ner"] != "O":
            return row["underthesea_ner"]
        else:
            return "O"

    df["final_label"] = df.apply(merge_labels, axis=1)

    # === 4️⃣ Convert to BIO format ===
    prev_label = "O"
    bio_labels = []
    for lbl in df["final_label"]:
        lbl = lbl.replace("B-", "").replace("I-", "")  # clean existing BIO prefixes
        if lbl == "O":
            bio_labels.append("O")
        elif lbl != prev_label:
            bio_labels.append(f"B-{lbl}")
        else:
            bio_labels.append(f"I-{lbl}")
        prev_label = lbl

    df["final_label"] = bio_labels

    # === 5️⃣ Return DataFrame ===
    df = df[["index", "form", "pos", "vncorenlp_ner", "underthesea_ner", "final_label"]]
    return df

In [6]:
df = pd.DataFrame(columns=["index", "form", "pos", "vncorenlp_ner", "underthesea_ner", "final_label"])

In [7]:
folder_path = "D:/Study/Education/Projects/Group_Project/source/data/annotate_text"
files = os.listdir(folder_path)

for file in files:
    with open(os.path.join(folder_path, file), 'r', encoding='utf-8') as f:
        docs = f.read()
        
        df = pd.concat([df, annotate_legal_entities(docs)], axis = 0)

In [8]:
df

,index,form,pos,vncorenlp_ner,underthesea_ner,final_label
0,1,QUỐC HỘI,I,O,O,O
1,2,-,CH,O,O,O
2,3,-,CH,O,O,O
3,4,-,CH,O,O,O
4,5,-,CH,O,O,O
...,...,...,...,...,...,...
319,320,Tài chính,N,I-ORG,O,I-ORG
320,321,trong,E,O,O,O
321,322,lĩnh vực,N,O,O,O
322,323,thuế,N,O,O,O


In [10]:
write_df_to_gs(df, 'test1')

'DataFrame appended to existing Google Sheet tab: test1'

In [5]:
from underthesea import sent_tokenize, word_tokenize

tokenize_sent = sent_tokenize(docs)

In [6]:
word_tokenize(tokenize_sent[0])

['BỘ',
 'TÀI CHÍNH',
 '-------',
 'CỘNG HÒA',
 'XÃ HỘI CHỦ NGHĨA VIỆT NAM',
 'Độc lập',
 '-',
 'Tự do',
 '-',
 'Hạnh phúc',
 '--------------- Số',
 ':',
 '52/2024',
 '/',
 'TT',
 '-',
 'BTC',
 'Hà Nội',
 ',',
 'ngày',
 '23',
 'tháng',
 '7',
 'năm',
 '2024',
 'THÔNG TƯ',
 'BÃI BỎ',
 'MỘT SỐ',
 'THÔNG TƯ CỦA',
 'BỘ TRƯỞNG',
 'BỘ',
 'TÀI CHÍNH',
 'TRONG',
 'LĨNH VỰC',
 'THUẾ',
 'Căn cứ',
 'Luật Ban hành văn bản',
 'quy phạm pháp luật',
 'ngày',
 '22',
 'tháng',
 '6',
 'năm',
 '2015',
 ';',
 'Luật',
 'sửa đổi',
 ',',
 'bổ sung',
 'một số',
 'điều',
 'của',
 'Luật',
 'Ban hành văn bản',
 'quy phạm pháp luật',
 'ngày',
 '18',
 'tháng',
 '6',
 'năm',
 '2020',
 ';',
 'Căn cứ',
 'Luật',
 'thuế giá trị',
 'gia tăng',
 'ngày',
 '03',
 'tháng',
 '6',
 'năm',
 '2008',
 ',',
 'Luật',
 'sửa đổi',
 ',',
 'bổ sung',
 'một số',
 'điều',
 'của',
 'Luật',
 'thuế giá trị',
 'gia tăng',
 'ngày',
 '19',
 'tháng',
 '6',
 'năm',
 '2013',
 'và',
 'các',
 'văn bản',
 'hướng dẫn',
 'thi hành',
 ';',
 'Căn cứ',
 '

In [7]:
import unicodedata

try_unicode = unicodedata.normalize('NFC', tokenize_sent[0])
try_unicode2 = unicodedata.normalize('NFC', tokenize_sent[-1])

In [8]:
df = annotate_legal_entities(try_unicode)
df2 = annotate_legal_entities(try_unicode2)

df = pd.concat([df, df2], axis = 0)

In [9]:
df

,index,form,pos,vncorenlp_ner,rule_label,underthesea_ner,final_label
0,1,BỘ,Nc,O,O,O,O
1,2,TÀI CHÍNH,Np,O,O,O,O
2,3,-,CH,O,O,O,O
3,4,-,CH,O,O,O,O
4,5,-,CH,O,O,O,O
...,...,...,...,...,...,...,...
322,323,thuế,N,O,O,O,O
323,324,.,CH,O,O,O,O
0,1,BỘ TRƯỞNG,Ny,O,O,O,O
1,2,THỨ TRƯỞNG,Ny,O,O,B-LOC,B-LOC


In [16]:
def export_bio(df, output_path="D:/Study/Education/Projects/Group_Project/source/data/ner/legal_ner_dataset.condll"):
    """
    Export annotated DataFrame to BIO text format for PhoBERT fine-tuning,
    keeping the column names at the top.
    """
    with open(output_path, "w", encoding="utf-8") as f:
        # Write header (column names)
        f.write("form final_label\n")

        # Write each token-label pair
        for _, row in df.iterrows():
            token = row["form"].replace(" ", "_")
            label = row["final_label"]
            f.write(f"{token} {label}\n")

        # Separate sentences if applicable (optional blank line)
        f.write("\n")

    print(f"✅ Saved BIO dataset to: {output_path}")

In [17]:
export_bio(df)

✅ Saved BIO dataset to: D:/Study/Education/Projects/Group_Project/source/data/ner/legal_ner_dataset.condll


In [ ]:
def read_conll(filepath):
    sentences, labels = [], []
    sent, lab = [], []
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:  # end of sentence
                if sent:
                    sentences.append(sent)
                    labels.append(lab)
                    sent, lab = [], []
                continue
            token, tag = line.split()
            sent.append(token)
            lab.append(tag)
    return sentences, labels

train_sents, train_labels = read_conll("train.conll")
val_sents, val_labels = read_conll("valid.conll")
test_sents, test_labels = read_conll("test.conll")

In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer

# dataset = load_dataset("text", data_files={"train": "D:/Study/Education/Projects/Group_Project/source/data/ner/legal_ner_dataset.txt"})

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

from datasets import Dataset

def read_conll_ner_file(file_path):
    tokens, labels = [], []
    all_tokens, all_labels = [], []
    
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens:  # end of sentence
                    all_tokens.append(tokens)
                    all_labels.append(labels)
                    tokens, labels = [], []
            else:
                parts = line.split()
                if len(parts) >= 2:
                    token, label = parts[0], parts[-1]
                    tokens.append(token)
                    labels.append(label)
        # Add last sentence if no trailing newline
        if tokens:
            all_tokens.append(tokens)
            all_labels.append(labels)
    
    return {"tokens": all_tokens, "ner_tags": all_labels}

dataset_dict = read_conll_ner_file("D:/Study/Education/Projects/Group_Project/source/data/ner/legal_ner_dataset.txt")
dataset = Dataset.from_dict(dataset_dict)
print(dataset)

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 1
})


Import data to BIO format to pretrain PhoBERT

In [21]:
annotator = VnCoreNLP(
    "D:/Study/Education/Projects/Group_Project/VnCoreNLP/VnCoreNLP-1.1.1.jar", 
    annotators="wseg,pos", 
    max_heap_size='-Xmx2g'
)

2025-10-05 20:59:58,805|INFO|vncorenlp.vncorenlp|Starting server on: http://127.0.0.1:50329
2025-10-05 20:59:58,833|INFO|vncorenlp.vncorenlp|Server ID: 15928
2025-10-05 21:00:00,869|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-05 21:00:07,900|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-05 21:00:14,941|INFO|vncorenlp.vncorenlp|Waiting until the server is available...
2025-10-05 21:00:21,319|INFO|vncorenlp.vncorenlp|The server is now available on: http://127.0.0.1:50329


In [22]:
text = unicodedata.normalize('NFC', tokenize_sent[0])

word_segmented_text1 = annotator.tokenize(text) 

In [23]:
word_segmented_text1

[['BỘ',
  'TÀI_CHÍNH',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  'CỘNG',
  'HÒA',
  'XÃ_HỘI_CHỦ_NGHĨA_VIỆT_NAM',
  'Độc_lập',
  '-',
  'Tự_do',
  '-',
  'Hạnh_phúc',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  'Số',
  ':',
  '52/2024/TT',
  '-',
  'BTC',
  'Hà_Nội',
  ',',
  'ngày',
  '23',
  'tháng',
  '7',
  'năm',
  '2024',
  'THÔNG_TƯ',
  'BÃI_BỎ',
  'MỘT_SỐ',
  'THÔNG_TƯ',
  'CỦA',
  'BỘ_TRƯỞNG',
  'BỘ',
  'TÀI_CHÍNH',
  'TRONG',
  'LĨNH_VỰC',
  'THUẾ',
  'Căn_cứ',
  'Luật',
  'Ban_hành',
  'văn_bản',
  'quy_phạm_pháp_luật',
  'ngày',
  '22',
  'tháng',
  '6',
  'năm',
  '2015',
  ';',
  'Luật',
  'sửa_đổi',
  ',',
  'bổ_sung',
  'một_số',
  'điều',
  'của',
  'Luật',
  'Ban_hành',
  'văn_bản',
  'quy_phạm_pháp_luật',
  'ngày',
  '18',
  'tháng',
  '6',
  'năm',
  '2020',
  ';',
  'Căn_cứ',
  'Luật',
  'thuế_giá_trị',
  'gia_tăng',
  'ngày',
  '03',
  'tháng',
  '6',
  'năm',
  '2008',
  ',',
  'Luật',
  'sửa_

In [7]:
## Normal form D (canonical decomposition and translate each character into its decomposed form)
text1 = unicodedata.normalize("NFC", "Thanh Thủy")
text2 = unicodedata.normalize("NFD", "Thanh Thủy")
## Normal form C (NFC applies canonical decomposition and then composes the pre-combined characters again)

# To perform word segmentation only
word_segmented_text1 = annotator1.tokenize(text1) 
word_segmented_text2 = annotator1.tokenize(text2)

print('Normal formm C')
print(*word_segmented_text1, sep="\n")

print('Normal formm D')
print(*word_segmented_text2, sep="\n")

Normal formm C
['Thanh_Thuỷ']
Normal formm D
['Thanh', 'Thủy']


In [6]:
text = get_text_from_s3('legaldocstorage/thong_tu_bai_bo_ve_thue_2024.pdf')

In [6]:
output['sentences']

[[{'index': 1, 'form': 'Vì', 'posTag': 'E', 'head': 0, 'depLabel': 'root'},
  {'index': 2, 'form': 'cô', 'posTag': 'N', 'head': 5, 'depLabel': 'sub'},
  {'index': 3, 'form': 'ấy', 'posTag': 'P', 'head': 2, 'depLabel': 'det'},
  {'index': 4, 'form': 'không', 'posTag': 'R', 'head': 5, 'depLabel': 'adv'},
  {'index': 5, 'form': 'đi', 'posTag': 'V', 'head': 1, 'depLabel': 'dep'},
  {'index': 6, 'form': 'học', 'posTag': 'V', 'head': 5, 'depLabel': 'vmod'},
  {'index': 7, 'form': 'nên', 'posTag': 'C', 'head': 5, 'depLabel': 'coord'},
  {'index': 8, 'form': 'cô', 'posTag': 'N', 'head': 10, 'depLabel': 'sub'},
  {'index': 9, 'form': 'ấy', 'posTag': 'P', 'head': 8, 'depLabel': 'det'},
  {'index': 10, 'form': 'trượt', 'posTag': 'V', 'head': 7, 'depLabel': 'conj'},
  {'index': 11,
   'form': 'đại_học',
   'posTag': 'N',
   'head': 10,
   'depLabel': 'dob'},
  {'index': 12, 'form': '.', 'posTag': 'CH', 'head': 7, 'depLabel': 'punct'}]]

In [2]:
rdrsegmenter = VnCoreNLP(
    "D:/Study/Education/Projects/Group_Project/VnCoreNLP/VnCoreNLP-1.1.1.jar", 
    annotators="wseg,pos,parse",  # word segmentation + POS + dependency parsing
    max_heap_size='-Xmx2g'
)

text = get_text_from_s3('legaldocstorage/quyet_dinh_ptrienhatang_2025.pdf')
text = sent_tokenize(text)[0]
# text = 'Vì cô ấy không đi học nên cô ấy trượt đại học.'

# Run the pipeline
output = rdrsegmenter.annotate(text)

# --- Print full dependency info with readable paths ---
for sent in output['sentences']:
    tokens = [tok['form'] for tok in sent]
    print(f"\nSentence: {' '.join(tokens)}")
    print("-" * 80)

    for tok in sent:
        head_id = tok['head']
        dep_rel = tok['depLabel']
        head_form = sent[head_id - 1]['form'] if head_id > 0 else 'ROOT'
        print(f"{tok['form']:>15}  ←[{dep_rel}]—  {head_form}")


Sentence: BỘ KHOA_HỌC VÀ CÔNG_NGHỆ - - - - - - - CỘNG HÒA XÃ_HỘI_CHỦ_NGHĨA_VIỆT_NAM Độc_lập - Tự_do - Hạnh_phúc - - - - - - - - - - - - - - - Số : 3004 / QĐ-BKHCN Hà_Nội , ngày 02 tháng 10 năm 2025 QUYẾT_ĐỊNH BAN_HÀNH KẾ_HOẠCH PHÁT_TRIỂN HẠ_TẦNG VIỄN_THÔNG , HẠ_TẦNG SỐ PHỤC_VỤ PHÁT_TRIỂN TRUNG_TÂM TÀI_CHÍNH QUỐC_TẾ TẠI VIỆT_NAM_BỘ_TRƯỞNG BỘ KHOA_HỌC VÀ CÔNG_NGHỆ Căn_cứ Nghị_định số 55/2025/NĐ-CP ngày 02 tháng 3 năm 2025 của Chính_phủ quy_định chức_năng , nhiệm_vụ , quyền_hạn và cơ_cấu tổ_chức của Bộ Khoa_học và Công_nghệ ; Căn_cứ Nghị_quyết số 259 / NQ-CP ngày 31 tháng 12 năm 2024 của Chính_phủ về Kế_hoạch hành_động triển_khai thực_hiện Thông_báo số 47 - TB / TW ngày 15 tháng 11 năm 2024 của Bộ_Chính_trị về việc xây_dựng Trung_tâm tài_chính khu_vực và quốc_tế tại Việt_Nam ; Căn_cứ Quyết_định số 1132 / QĐ-TTg ngày 09 tháng 10 năm 2024 của Thủ_tướng Chính_phủ phê_duyệt Chiến_lược hạ_tầng số đến năm 2025 và định_hướng đến năm 2030 ; Căn_cứ Quyết_định số 36 / QĐ-TTg ngày 11 tháng 01 năm 2

In [5]:
sent = output["sentences"][0]

def find_sdp(sent, start_form, end_form):
    """
    Find shortest dependency path between two tokens in a parsed sentence.
    """
    # Map: id -> head id
    heads = {tok.get("index", tok.get("id")): tok["head"] for tok in sent}
    forms = {tok.get("index", tok.get("id")): tok["form"] for tok in sent}
    
    # Find token indices for start and end
    start_ids = [tok.get("index", tok.get("id")) for tok in sent if tok["form"] == start_form]
    end_ids = [tok.get("index", tok.get("id")) for tok in sent if tok["form"] == end_form]
    
    if not start_ids or not end_ids:
        print("Could not find both tokens in sentence.")
        return None
    
    start = start_ids[0]
    end = end_ids[0]
    
    # Function to get all ancestors of a node (including itself)
    def get_ancestors(node):
        path = [node]
        while heads[node] > 0:
            node = heads[node]
            path.append(node)
        return path
    
    # Get ancestors of both nodes
    start_anc = get_ancestors(start)
    end_anc = get_ancestors(end)
    
    # Find lowest common ancestor (LCA)
    lca = next((a for a in start_anc if a in end_anc), None)
    if lca is None:
        return None
    
    # Path = start → LCA → end
    start_path = start_anc[:start_anc.index(lca)+1]
    end_path = end_anc[:end_anc.index(lca)]
    end_path.reverse()
    full_path = start_path + end_path
    
    return [forms[i] for i in full_path]


# --- Try it out ---
sdp = find_sdp(sent, "Căn_cứ", "Nghị_quyết")
print("→ Shortest Dependency Path (SDP):")
print("  " + " → ".join(sdp))

→ Shortest Dependency Path (SDP):
  Căn_cứ → hành_động → Căn_cứ → Nghị_quyết


In [6]:
def get_subtree(tokens, head_index):
    children = {tok['head']: [] for tok in tokens}
    for tok in tokens:
        children.setdefault(tok['head'], []).append(tok['index'])

    def collect(idx):
        result = [tokens[idx - 1]['form']]
        for c in children.get(idx, []):
            result.extend(collect(c))
        return result

    return " ".join(collect(head_index))

for sent in output['sentences']:
    for tok in sent:
        if tok['form'].lower() == 'căn_cứ':
            print("📘 Legal reference:", get_subtree(sent, tok['index']))

📘 Legal reference: Căn_cứ BỘ KHOA_HỌC VÀ CÔNG_NGHỆ - - - - - - - - - - - - - - - - - - , CỘNG HÒA - XÃ_HỘI_CHỦ_NGHĨA_VIỆT_NAM Độc_lập - Tự_do - - Hạnh_phúc - Số - : 3004 / QĐ-BKHCN Hà_Nội , ngày 02 tháng 10 năm 2025 QUYẾT_ĐỊNH BAN_HÀNH KẾ_HOẠCH PHÁT_TRIỂN HẠ_TẦNG VIỄN_THÔNG VÀ HẠ_TẦNG SỐ PHỤC_VỤ PHÁT_TRIỂN TRUNG_TÂM TÀI_CHÍNH QUỐC_TẾ TẠI VIỆT_NAM_BỘ_TRƯỞNG BỘ KHOA_HỌC CÔNG_NGHỆ Nghị_định số 55/2025/NĐ-CP ngày 02 tháng 3 năm 2025 của Chính_phủ quy_định chức_năng nhiệm_vụ của Bộ Khoa_học và Công_nghệ về Kế_hoạch quyền_hạn , , và cơ_cấu tổ_chức ; hành_động Căn_cứ Nghị_quyết số 259 / NQ-CP ngày 31 tháng 12 năm 2024 của Chính_phủ triển_khai thực_hiện Thông_báo số 47 - TB / TW ngày 15 tháng 11 năm 2024 của Bộ_Chính_trị về việc xây_dựng Trung_tâm tài_chính khu_vực và quốc_tế tại Việt_Nam ; Quyết_định Căn_cứ số 1132 / QĐ-TTg ngày 09 tháng 10 năm 2024 của Thủ_tướng Chính_phủ phê_duyệt Chiến_lược hạ_tầng số đến năm 2025 và định_hướng đến năm 2030 ; Quyết_định Căn_cứ số 36 / QĐ-TTg ngày 11 tháng 